#ЛАБОРАТОРНА РОБОТА №7
#Завдання 1.

In [ ]:
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
import pandas as pd

# Завантаження датасету MovieLens 100k
data = Dataset.load_builtin('ml-100k')

# Створення pandas DataFrame для виводу перших 5 рядків
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])
print(df.head(5))


  user item  rating  timestamp
0  196  242     3.0  881250949
1  186  302     3.0  891717742
2   22  377     1.0  878887116
3  244   51     2.0  880606923
4  166  346     1.0  886397596


In [ ]:
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate

# Алгоритм SVD
algo_svd = SVD()
# Алгоритм KNN
algo_knn = KNNBasic()

# Крос-валідація для SVD
cv_svd = cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)
# Крос-валідація для KNN
cv_knn = cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)


Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7365  0.7405  0.7372  0.7370  0.7341  0.7371  0.0020  
Fit time          1.70    1.37    1.39    1.92    1.51    1.58    0.21    
Test time         0.18    0.12    0.11    0.19    0.11    0.14    0.04    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7774  0.7739  0.7693  0.7764  0.7693  0.7733  0.0034  
Fit time          0.34    0.38    0.35    0.35    0.37    0.36    0.01    
Test time

In [ ]:
# Обчислення середнього значення MAE
mae_svd = sum(cv_svd['test_mae']) / len(cv_svd['test_mae'])
mae_knn = sum(cv_knn['test_mae']) / len(cv_knn['test_mae'])

print(f"MAE для SVD: {mae_svd}")
print(f"MAE для KNN: {mae_knn}")

# Вибір найкращого алгоритму
best_algo = algo_svd if mae_svd < mae_knn else algo_knn
print(f"Найкращий алгоритм: {'SVD' if best_algo == algo_svd else 'KNNBasic'}")


MAE для SVD: 0.7370619244695669
MAE для KNN: 0.7732619532852764
Найкращий алгоритм: SVD


In [ ]:
# Тренування найкращого алгоритму
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Отримання рекомендацій для користувача (наприклад, користувач з ID 196)
user_id = str(196)
items = df['item'].unique()

# Фільми, які користувач ще не оцінив
items_to_rate = [item for item in items if not trainset.knows_item(item)]

# Прогнозування рейтингів та сортування за значенням
predictions = [best_algo.predict(user_id, item) for item in items_to_rate]
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Виведення топ-10 рекомендацій
print("Рекомендовані фільми для користувача 196:")
for rec in recommendations:
    print(f"Фільм ID: {rec.iid}, Прогнозований рейтинг: {rec.est}")


Рекомендовані фільми для користувача 196:
Фільм ID: 127, Прогнозований рейтинг: 4.679716979466631
Фільм ID: 408, Прогнозований рейтинг: 4.656020265713531
Фільм ID: 64, Прогнозований рейтинг: 4.65585164490117
Фільм ID: 50, Прогнозований рейтинг: 4.619729296095811
Фільм ID: 318, Прогнозований рейтинг: 4.6096196513170025
Фільм ID: 511, Прогнозований рейтинг: 4.577168455479542
Фільм ID: 515, Прогнозований рейтинг: 4.542916570162102
Фільм ID: 483, Прогнозований рейтинг: 4.533641256066035
Фільм ID: 357, Прогнозований рейтинг: 4.52327326197612
Фільм ID: 98, Прогнозований рейтинг: 4.523147032169898


#Завдання 2.

In [ ]:
from collections import defaultdict

# Функція для створення власної рекомендаційної системи
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортування прогнозів для кожного користувача та вибір топ-N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Використання крос-валідації для отримання рекомендацій
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Прогнозування для тестової вибірки
testset = trainset.build_anti_testset()
predictions = best_algo.test(testset)

# Отримання топ-10 рекомендацій для всіх користувачів
top_n = get_top_n(predictions, n=10)

# Виведення рекомендацій для конкретного користувача (наприклад, користувач 196)
print(f"Рекомендації для користувача 196: {top_n['196']}")


Рекомендації для користувача 196: [('190', 4.604231887869725), ('223', 4.477210881399889), ('169', 4.476833336828952), ('318', 4.474242734193191), ('98', 4.450175275487304), ('64', 4.388061303399808), ('79', 4.371319590166469), ('647', 4.370901090573178), ('480', 4.364751191148627), ('603', 4.3625351144557065)]


Висновок: Під час лабораторної роботи було досліджено два підходи до створення рекомендаційних систем: використання алгоритмів із бібліотеки Surprise та розробка власної рекомендаційної системи. Проаналізовано кілька алгоритмів, таких як SVD та KNN, з оцінкою їхньої точності за допомогою метрики MAE, і визначено найкращий варіант для використання. На основі цього алгоритму було створено рекомендації для конкретного користувача.

